<a href="https://colab.research.google.com/github/SasithWickrama/Enterprise_Mobility/blob/main/fccGenrateImageUsingCaption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Reshape, Conv2DTranspose, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from PIL import Image

# Mount Google Drive (if needed)
from google.colab import drive
drive.mount('/content/drive')

# Set base directory
BASE_DIR = '/content/drive/MyDrive/kaggle/input/flickr8k/Images'

# Load captions
captions = [
    "A baby girl in an orange dress gets wet as she stands next to a water sprinkler",
    "A blonde toddler wearing an orange dress is wet and standing beside a sprinkler in a yard",
    "A child in a dress is looking at a sprinkler",
    "A little girl in an orange dress is running through the sprinkler in the yard",
    "On wet grass, little blond girl in orange dress plays in sprinkler"
]

# Tokenize captions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions)
vocab_size = len(tokenizer.word_index) + 1

# Generate sequences from captions
sequences = tokenizer.texts_to_sequences(captions)
max_seq_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

# Define the text-to-image generation model
text_to_image_model = Sequential([
    Embedding(vocab_size, 50, input_length=max_seq_length),
    LSTM(256, return_sequences=False),
    Dense(512 * 512 * 1, activation='relu'),  # Adjust units to match desired output shape
    Reshape((512, 512, 1)),  # Reshape to match image shape
    Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', activation='relu'),  # Upsample
    BatchNormalization(),
    Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', activation='relu'),  # Upsample
    BatchNormalization(),
    Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', activation='sigmoid')  # Final output
])

# Compile the model
optimizer = Adam(learning_rate=0.001)
text_to_image_model.compile(loss='mse', optimizer=optimizer)

# Define callbacks
checkpoint = ModelCheckpoint('text_to_image_model.h5', save_best_only=True)
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.2, patience=2)

# Define target data (images)
target_data = np.zeros((len(padded_sequences), 4096, 4096, 1))

# Train the model
text_to_image_model.fit(padded_sequences, target_data, epochs=50, batch_size=1, callbacks=[checkpoint, early_stopping, reduce_lr])

# Generate images from captions
generated_images = text_to_image_model.predict(padded_sequences)

# Save generated images
output_dir = os.path.join(BASE_DIR, 'Generated_Images')
os.makedirs(output_dir, exist_ok=True)
for i, image_array in enumerate(generated_images):
    image = Image.fromarray((image_array[:, :, 0] * 255).astype(np.uint8))
    image.save(os.path.join(output_dir, f'image_{i+1}.png'))

print("Images generated successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50
5/5 [==============================] - ETA: 0s - loss: 0.2486 

5/5 [==============================] - 152s 30s/step - loss: 0.2486 - lr: 0.0010
Epoch 2/50
5/5 [==============================] - ETA: 0s - loss: 0.2437 

5/5 [==============================] - 143s 29s/step - loss: 0.2437 - lr: 0.0010
Epoch 3/50
5/5 [==============================] - ETA: 0s - loss: 0.2363 

5/5 [==============================] - 144s 29s/step - loss: 0.2363 - lr: 0.0010
Epoch 4/50
5/5 [==============================] - ETA: 0s - loss: 0.2260 

5/5 [==============================] - 144s 29s/step - loss: 0.2260 - lr: 0.0010
Epoch 5/50
5/5 [==============================] - ETA: 0s - loss: 0.2126 

...

5/5 [==============================] - 144s 28s/step - loss: 0.0016 - lr: 0.0010
Epoch 47/50
5/5 [==============================] - ETA: 0s - loss: 0.0015 

5/5 [==============================] - 143s 29s/step - loss: 0.0015 - lr: 0.0010
Epoch 48/50
5/5 [==============================] - ETA: 0s - loss: 0.0014 

5/5 [==============================] - 142s 28s/step - loss: 0.0014 - lr: 0.0010
Epoch 49/50
5/5 [==============================] - ETA: 0s - loss: 0.0014 

5/5 [==============================] - 145s 29s/step - loss: 0.0014 - lr: 0.0010
Epoch 50/50
5/5 [==============================] - ETA: 0s - loss: 0.0013 

1/1 [==============================] - 35s 35s/step
Images generated successfully!
